# Zero to Hero

## 1.15 Montecarlo Estimation con una función en loop

El objetivo es implementar con un for loop la estimacion montecarlo, que promedia las ganancias

In [4]:
options(warn=-1)

In [5]:
rm(list=ls())
gc()

library("data.table")   #cargo la libreria  data.table
library("rpart")  #cargo la libreria  rpart
library("rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1941893,103.8,3661371,195.6,3661371,195.6
Vcells,3242602,24.8,8388608,64.0,5255196,40.1


In [6]:
options(repr.plot.width = 20, repr.plot.height = 10) 
setwd("C:\\Users\\oliva\\OneDrive\\Desktop\\DS\\Austral\\08 - Labo1") #Aqui se debe poner la ruta de la PC local

La funcion que definimos en el notebook anterior

In [15]:
GananciaArbol  <-  function(semilla, data, x, train = 0.70) {
  #establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p = 0.70, list = FALSE)
  
  modelo <- rpart(formula= "clase_ternaria ~ ." ,
                          data= data[ train_rows],  #los datos de training
                          xval= 0, #no CV
                          control= x)
    
  #Aplico el modelo a los datos de testing  
  prediccion  <- predict( modelo, data[ -train_rows], type = "prob")
  
  prob_baja2  <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[ -train_rows, sum((prob_baja2>1/60) * ifelse( clase_ternaria == "BAJA+2", 59000, -1000))]
  
  #normalizo la ganancia
  ganancia_testing_normalizada <- ganancia_testing/0.3  

  return(ganancia_testing_normalizada)
}

In [16]:
ArbolMontecarlo <- function(semillas, data, x, train = 0.70){
    
  vector_ganancias <- c()  #vector donde voy a ir acumulando las ganancias
  for(semilla in ksemillas)
  {
    ganancia <- GananciaArbol(semilla, dataset, x = x, train = 0.70 )
    vector_ganancias <- c(vector_ganancias, ganancia)
  }
    
  return(mean(vector_ganancias))
}

In [17]:
dataset <- fread("./labo/datasets/paquete_premium_202011.csv")   #cargo el dataset

In [18]:
ksemillas <- c(477293, 525713, 679561, 895627, 978643) #reemplazar por las propias semillas

In [19]:
#defino el primer set de hiperparametros
param1  <- list("cp"= -0.5,
                "minsplit"=  900,
                "minbucket"= 440,
                "maxdepth"= 5 )

Llamo a la fucion ArbolMontecarlo con las 5 semillas de ksemillas

In [20]:
ganancia_montecarlo1  <- ArbolMontecarlo(ksemillas, dataset, x = param1, train= 0.70 )

In [21]:
#el SEGUNDO set de hiperparametros
param2  <- list("cp"= -0.5,
                "minsplit"=  1340,
                "minbucket"=  600,
                "maxdepth"= 6 )

In [22]:
ganancia_montecarlo2  <- ArbolMontecarlo( ksemillas, dataset, x = param2, train = 0.70 )

In [23]:
ganancia_montecarlo1
ganancia_montecarlo2

[1] 10050667

[1] 9944667

Segun este experimento, el primer juego de hiperparametros es mejor que el primero
<br>
¿Se cumplirá esto subiendo a Kaggle?
<br>
¿Qué sucede si en Kaggle funciona bastante mejor el segundo set de hiperparámetros?